In [233]:
import gurobipy as gp
from gurobipy import GRB

In [234]:
num_periods = 7
num_items = 1
num_suppliers = 3
num_provider_warehouses = 4
num_retailer_warehouses = 1
num_emergency_warehouses = 1
num_commitment_durations = 5
num_scenarios = 3

In [235]:
T = list(range(1, num_periods + 1))  # Time periods 
I = [f'item{i + 1}' for i in range(num_items)]  # Items 
J = [f's{j + 1}' for j in range(num_suppliers)]  # Suppliers 
K = [f'k{k + 1}' for k in range(num_provider_warehouses)]  # Provider warehouses 
R = [f'r{r + 1}' for r in range(num_retailer_warehouses)]  # Retailer warehouses 
E = [f'e{e + 1}' for e in range(num_emergency_warehouses)]  # Emergency warehouses
M = list(range(1, num_commitment_durations + 1))  # Commitment durations 
Omega = [f'omega{o + 1}' for o in range(num_scenarios)]  # Scenarios 

p = {omega: 1.0 / len(Omega) for omega in Omega} #probability

In [236]:
# Costs and Fixed Parameters
h_r = {i: 0.2 for i in I}  # Inventory holding costs for retailer 
h_k = {i: 0.2 for i in I}  # Inventory holding costs for provider 
h_e = {i: 0.2 for i in I}  # Inventory holding costs for emergency 

b = {i: 1.5 for i in I}  # Delivery costs 

beta = {i: 5 for i in I}  # Lost sales costs 

delta = {i: 2 for i in I}  # Unaccepted returns costs 

c_r = {j: 1.0 for j in J}  # Transportation costs to retailer 
c_k = {j: 1.0 for j in J}  # Transportation costs to provider 
c_e = {j: 1.0 for j in J}  # Transportation costs to emergency 

c_d_r = 1.5  # Disposal costs from retailer 
c_d_k = 1.5  # Disposal costs from provider 
c_d_e = 1.5  # Disposal costs from emergency 

c_ref = {i: 3 for i in I}  # Refurbishment costs 
theta = {i: 1 for i in I}  # Disposal fees 

F = {j: 1000 for j in J}  # Supplier investment costs 
alpha = 500  # Commitment cost 
gamma = 0.9  # Discount factor 

# Capacities 
C_r = {r: 5000 for r in R}  # Retailer warehouse capacity
C_k = {k: 5000 for k in K}  # Provider warehouse capacity
C_e = {e: 5000 for e in E}  # Emergency warehouse capacity

# Refurbishing percentage 
phi = {(i, t, omega): 0.2 for i in I for t in T for omega in Omega}

# Demand 
D = {(i, t, omega): 100 for i in I for t in T for omega in Omega}

# Lead time (Ld in images - kept scalar as it appeared without indices in the image)
Ld = 1 

# Return percentage (rho_it_omega in images)
rho = {(i, t, omega): 0.1 for i in I for t in T for omega in Omega}

# Supply capacity (S_ijt_omega in images)
S = {(i, j, t, omega): 1000 for i in I for j in J for t in T for omega in Omega}

# Disposal capacity (DC_it_omega in images)
DC = {(i, t, omega): 200 for i in I for t in T for omega in Omega}

In [237]:
model = gp.Model("SupplyChainOptimization")

In [238]:
# First Stage Variables 
y = model.addVars(J, vtype=GRB.BINARY, name="y")
g = model.addVars(K, M, T, vtype=GRB.BINARY, name="g")
r = model.addVars(K, M, T, vtype=GRB.BINARY, name="r")


In [239]:
# Second Stage Variables 

# Forward inventory levels (vf: v_f in images)
vf_r = model.addVars(I, R, T, Omega, name="vf_r")
vf_k = model.addVars(I, K, T, Omega, name="vf_k")
vf_e = model.addVars(I, E, T, Omega, name="vf_e")

# Disposable inventory levels (vd: v_d in images)
vd_r = model.addVars(I, R, T, Omega, name="vd_r")
vd_k = model.addVars(I, K, T, Omega, name="vd_k")
vd_e = model.addVars(I, E, T, Omega, name="vd_e")

# Returns and disposal quantities
# n_...: Quantity of item i returned to respective warehouse
n_r = model.addVars(I, R, T, Omega, name="n_r")
n_k = model.addVars(I, K, T, Omega, name="n_k")
n_e = model.addVars(I, E, T, Omega, name="n_e")

# f_...: Quantity of item i returned to respective warehouse which will be disposed
f_r = model.addVars(I, R, T, Omega, name="f_r")
f_k = model.addVars(I, K, T, Omega, name="f_k")
f_e = model.addVars(I, E, T, Omega, name="f_e")

# a_...: Quantity of item i returned from respective warehouse to disposal facility
a_r = model.addVars(I, R, T, Omega, name="a_r")
a_k = model.addVars(I, K, T, Omega, name="a_k")
a_e = model.addVars(I, E, T, Omega, name="a_e")

# Refurbishing quantities
q_r = model.addVars(I, R, T, Omega, name="q_r")
q_k = model.addVars(I, K, T, Omega, name="q_k")
q_e = model.addVars(I, E, T, Omega, name="q_e")

# Transportation quantities (x_ijt_...)
x_r = model.addVars(I, J, R, T, Omega, name="x_r") # to retailer warehouse
x_k = model.addVars(I, J, K, T, Omega, name="x_k") # to provider warehouse
x_e = model.addVars(I, J, E, T, Omega, name="x_e") # to emergency warehouse

# Delivery quantities (u_...): to satisfy demand
u_r = model.addVars(I, R, T, Omega, name="u_r")
u_k = model.addVars(I, K, T, Omega, name="u_k")
u_e = model.addVars(I, E, T, Omega, name="u_e")

# Lost sales and unaccepted returns
z = model.addVars(I, T, Omega, name="z") # z_it_omega
w = model.addVars(I, T, Omega, name="w") # w_it_omega


In [240]:
# First stage costs 
# Sum of supplier investment costs + sum of commitment costs over all periods
first_stage_cost = (
    gp.quicksum(F[j] * y[j] for j in J) +
    gp.quicksum(m * alpha * (gamma**m) * g[k, m, t] for k in K for m in M for t in T)
)

# Second stage costs 
second_stage_cost = gp.quicksum(
    p[omega] * (
        # Holding costs for forward inventory
        gp.quicksum(h_r[i] * (vf_r[i, r, t, omega]) for i in I for r in R for t in T) +
        gp.quicksum(h_k[i] * (vf_k[i, k, t, omega]) for i in I for k in K for t in T) +
        gp.quicksum(h_e[i] * (vf_e[i, e, t, omega]) for i in I for e in E for t in T) +

        # Holding costs for disposable inventory
        gp.quicksum(h_r[i] * vd_r[i, r, t, omega] for i in I for r in R for t in T) +
        gp.quicksum(h_k[i] * vd_k[i, k, t, omega] for i in I for k in K for t in T) +
        gp.quicksum(h_e[i] * vd_e[i, e, t, omega] for i in I for e in E for t in T) +
        
        # Delivery costs for forward flow (u) and return flow (n)
        gp.quicksum(b[i] * (u_r[i, r, t, omega] + n_r[i, r, t, omega]) for i in I for r in R for t in T) +
        gp.quicksum(b[i] * (u_k[i, k, t, omega] + n_k[i, k, t, omega]) for i in I for k in K for t in T) +
        gp.quicksum(b[i] * (u_e[i, e, t, omega] + n_e[i, e, t, omega]) for i in I for e in E for t in T) +
        
        # Penalty costs for lost sales (z) and unaccepted returns (w)
        gp.quicksum(beta[i] * z[i, t, omega] for i in I for t in T) +
        gp.quicksum(delta[i] * w[i, t, omega] for i in I for t in T) +
        
        # Transportation costs from suppliers to warehouses
        gp.quicksum(c_r[j] * x_r[i, j, r_val, t, omega] for i in I for j in J for r_val in R for t in T) +
        gp.quicksum(c_k[j] * x_k[i, j, k_val, t, omega] for i in I for j in J for k_val in K for t in T) +
        gp.quicksum(c_e[j] * x_e[i, j, e_val, t, omega] for i in I for j in J for e_val in E for t in T) +
        
        # Disposal costs from warehouses to disposal facility
        c_d_r * gp.quicksum(a_r[i, r_val, t, omega] for i in I for r_val in R for t in T) +
        c_d_k * gp.quicksum(a_k[i, k_val, t, omega] for i in I for k_val in K for t in T) +
        c_d_e * gp.quicksum(a_e[i, e_val, t, omega] for i in I for e_val in E for t in T) +
        
        # Refurbishment costs
        gp.quicksum(c_ref[i] * q_r[i, r_val, t, omega] for i in I for r_val in R for t in T) +
        gp.quicksum(c_ref[i] * q_k[i, k_val, t, omega] for i in I for k_val in K for t in T) +
        gp.quicksum(c_ref[i] * q_e[i, e_val, t, omega] for i in I for e_val in E for t in T) +
        
        # Disposal fees 
        gp.quicksum(theta[i] * a_r[i, r_val, t, omega] for i in I for r_val in R for t in T) +
        gp.quicksum(theta[i] * a_k[i, k_val, t, omega] for i in I for k_val in K for t in T) +
        gp.quicksum(theta[i] * a_e[i, e_val, t, omega] for i in I for e_val in E for t in T)
    )
    for omega in Omega
)


In [241]:
model.setObjective(first_stage_cost + second_stage_cost, GRB.MINIMIZE)

In [242]:
# First Stage Constraints 

for k in K:
    for m in M:
        for t in T:
            # Constraint (1)
            end_period = min(t + m - 1, T[-1])
            if t <= end_period:
                model.addConstr(
                    gp.quicksum(g[k, m, tau] for tau in range(t, end_period + 1)) <= 1,
                    f"commitment_once_C1_{k}_{m}_{t}"
                )
            
            # Constraint (2)
            start_period = max(t - m + 1, T[0])
            if start_period <= t:
                model.addConstr(
                    gp.quicksum(g[k, m, tau] for tau in range(start_period, t + 1)) == r[k, m, t],
                    f"link_g_r_C2_{k}_{m}_{t}"
                )

for k in K:
    for t in T:
        # Constraint (3)
        model.addConstr(
            gp.quicksum(r[k, m, t] for m in M) <= 1,
            f"one_active_commitment_C3_{k}_{t}"
        )
        
        # Constraint (4)
        model.addConstr(
            gp.quicksum(g[k, m, t] for m in M) <= 1,
            f"one_new_commitment_C4_{k}_{t}"
        )

sum_active_commitments_r_precalculated = {}
for k in K:
    for t in T:
        r_vars_to_sum = []
        for m in M:
            if (k, m, t) in r.keys():
                r_vars_to_sum.append(r[k, m, t])
        sum_active_commitments_r_precalculated[k, t] = gp.quicksum(r_vars_to_sum)



In [243]:
# Second Stage Constraints 

for omega in Omega:
    # Constraint (7)
    for i in I:
        for j in J:
            for t in T:
                model.addConstr(
                    gp.quicksum(x_r[i, j, r_val, t, omega] for r_val in R) +
                    gp.quicksum(x_k[i, j, k_val, t, omega] for k_val in K) +
                    gp.quicksum(x_e[i, j, e_val, t, omega] for e_val in E) <= S[i, j, t, omega] * y[j],
                    f"supply_C7_{i}_{j}_{t}_{omega}"
                )
    
    # Constraint (8)
    for i in I:
        for t in T:
            model.addConstr(
                gp.quicksum(a_r[i, r_val, t, omega] for r_val in R) +
                gp.quicksum(a_k[i, k_val, t, omega] for k_val in K) +
                gp.quicksum(a_e[i, e_val, t, omega] for e_val in E) <= DC[i, t, omega],
                f"disposal_cap_C8_{i}_{t}_{omega}"
            )
    
    # Inventory balance constraints (Forward Flow)
    for i in I:
        # Constraint (9)
        for r_val in R:
            for t in T:
                if t == T[0]: 
                    model.addConstr(
                        u_r[i, r_val, t, omega] + vf_r[i, r_val, t, omega] == 
                        gp.quicksum(x_r[i, j_val, r_val, t, omega] for j_val in J) + q_r[i, r_val, t, omega],
                        f"retailer_fwd_inv_balance_init_C9_{i}_{r_val}_{t}_{omega}"
                    )
                else:
                    model.addConstr(
                        u_r[i, r_val, t, omega] + vf_r[i, r_val, t, omega] == 
                        vf_r[i, r_val, t-1, omega] + gp.quicksum(x_r[i, j_val, r_val, t, omega] for j_val in J) + q_r[i, r_val, t, omega],
                        f"retailer_fwd_inv_balance_C9_{i}_{r_val}_{t}_{omega}"
                    )
        
        # Constraint (10)
        for k_val in K:
            for t in T:
                if t == T[0]:
                    model.addConstr(
                        u_k[i, k_val, t, omega] + vf_k[i, k_val, t, omega] == 
                        gp.quicksum(x_k[i, j_val, k_val, t, omega] for j_val in J) + q_k[i, k_val, t, omega],
                        f"provider_fwd_inv_balance_init_C10_{i}_{k_val}_{t}_{omega}"
                    )
                else:
                    model.addConstr(
                        u_k[i, k_val, t, omega] + vf_k[i, k_val, t, omega] == 
                        vf_k[i, k_val, t-1, omega] + gp.quicksum(x_k[i, j_val, k_val, t, omega] for j_val in J) + q_k[i, k_val, t, omega],
                        f"provider_fwd_inv_balance_C10_{i}_{k_val}_{t}_{omega}"
                    )
        
        # Constraint (11)
        for e_val in E:
            for t in T:
                if t == T[0]:
                    model.addConstr(
                        u_e[i, e_val, t, omega] + vf_e[i, e_val, t, omega] == 
                        gp.quicksum(x_e[i, j_val, e_val, t, omega] for j_val in J) + q_e[i, e_val, t, omega],
                        f"emergency_fwd_inv_balance_init_C11_{i}_{e_val}_{t}_{omega}"
                    )
                else:
                    model.addConstr(
                        u_e[i, e_val, t, omega] + vf_e[i, e_val, t, omega] == 
                        vf_e[i, e_val, t-1, omega] + gp.quicksum(x_e[i, j_val, e_val, t, omega] for j_val in J) + q_e[i, e_val, t, omega],
                        f"emergency_fwd_inv_balance_C11_{i}_{e_val}_{t}_{omega}"
                    )
    
    # Return flow constraints
    for i in I:
        # Constraint (12)
        for r_val in R:
            for t in T:
                model.addConstr(
                    n_r[i, r_val, t, omega] == f_r[i, r_val, t, omega] + q_r[i, r_val, t, omega],
                    f"retailer_total_returns_C12_{i}_{r_val}_{t}_{omega}"
                )
        
        # Constraint (14)
        for k_val in K:
            for t in T:
                model.addConstr(
                    n_k[i, k_val, t, omega] == f_k[i, k_val, t, omega] + q_k[i, k_val, t, omega],
                    f"provider_total_returns_C14_{i}_{k_val}_{t}_{omega}"
                )
        
        # Constraint (16)
        for e_val in E:
            for t in T:
                model.addConstr(
                    n_e[i, e_val, t, omega] == f_e[i, e_val, t, omega] + q_e[i, e_val, t, omega],
                    f"emergency_total_returns_C16_{i}_{e_val}_{t}_{omega}"
                )
    
    # Disposal inventory balance
    for i in I:
        # Constraint (13)
        for r_val in R:
            for t in T:
                if t == T[0]:
                    model.addConstr(
                        a_r[i, r_val, t, omega] + vd_r[i, r_val, t, omega] == f_r[i, r_val, t, omega],
                        f"retailer_disposal_inv_balance_init_C13_{i}_{r_val}_{t}_{omega}"
                    )
                else:
                    model.addConstr(
                        a_r[i, r_val, t, omega] + vd_r[i, r_val, t, omega] == 
                        vd_r[i, r_val, t-1, omega] + f_r[i, r_val, t, omega],
                        f"retailer_disposal_inv_balance_C13_{i}_{r_val}_{t}_{omega}"
                    )
        
        # Constraint (15)
        for k_val in K:
            for t in T:
                if t == T[0]:
                    model.addConstr(
                        a_k[i, k_val, t, omega] + vd_k[i, k_val, t, omega] == f_k[i, k_val, t, omega],
                        f"provider_disposal_inv_balance_init_C15_{i}_{k_val}_{t}_{omega}"
                    )
                else:
                    model.addConstr(
                        a_k[i, k_val, t, omega] + vd_k[i, k_val, t, omega] == 
                        vd_k[i, k_val, t-1, omega] + f_k[i, k_val, t, omega],
                        f"provider_disposal_inv_balance_C15_{i}_{k_val}_{t}_{omega}"
                    )
        
        # Constraint (17)
        for e_val in E:
            for t in T:
                if t == T[0]:
                    model.addConstr(
                        a_e[i, e_val, t, omega] + vd_e[i, e_val, t, omega] == f_e[i, e_val, t, omega],
                        f"emergency_disposal_inv_balance_init_C17_{i}_{e_val}_{t}_{omega}"
                    )
                else:
                    model.addConstr(
                        a_e[i, e_val, t, omega] + vd_e[i, e_val, t, omega] == 
                        vd_e[i, e_val, t-1, omega] + f_e[i, e_val, t, omega],
                        f"emergency_disposal_inv_balance_C17_{i}_{e_val}_{t}_{omega}"
                    )
    
    # Constraint (18)
    for i in I:
        for t in T:
            model.addConstr(
                gp.quicksum(q_r[i, r_val, t, omega] for r_val in R) +
                gp.quicksum(q_k[i, k_val, t, omega] for k_val in K) +
                gp.quicksum(q_e[i, e_val, t, omega] for e_val in E) == 
                phi[i, t, omega] * (
                    gp.quicksum(n_r[i, r_val, t, omega] for r_val in R) +
                    gp.quicksum(n_k[i, k_val, t, omega] for k_val in K) +
                    gp.quicksum(n_e[i, e_val, t, omega] for e_val in E)
                ),
                f"refurbishing_C18_{i}_{t}_{omega}"
            )
    
    # Constraint (19)
    for i in I:
        for t in T:
            delivery_sum = 0
            if (t - Ld) in T:
                delivery_sum += gp.quicksum(u_r[i, r_val, t - Ld, omega] for r_val in R)
                delivery_sum += gp.quicksum(u_k[i, k_val, t - Ld, omega] for k_val in K)
                delivery_sum += gp.quicksum(u_e[i, e_val, t - Ld, omega] for e_val in E)

            model.addConstr(
                delivery_sum + z[i, t, omega] >= D[i, t, omega],
                f"demand_C19_{i}_{t}_{omega}"
            )

    # Constraint (20)
    for i in I:
        for t in T:
            lhs_sum_n = 0
            if (t - Ld) in T:
                lhs_sum_n += gp.quicksum(n_r[i, r_val, t - Ld, omega] for r_val in R)
                lhs_sum_n += gp.quicksum(n_k[i, k_val, t - Ld, omega] for k_val in K)
                lhs_sum_n += gp.quicksum(n_e[i, e_val, t - Ld, omega] for e_val in E)

            rhs_sum_u = 0
            if (t - Ld - 1) in T:
                rhs_sum_u += gp.quicksum(u_r[i, r_val, t - Ld - 1, omega] for r_val in R)
                rhs_sum_u += gp.quicksum(u_k[i, k_val, t - Ld - 1, omega] for k_val in K)
                rhs_sum_u += gp.quicksum(u_e[i, e_val, t - Ld - 1, omega] for e_val in E)
            
            model.addConstr(
                lhs_sum_n + w[i, t, omega] == rho[i, t, omega] * rhs_sum_u,
                f"return_flow_C20_{i}_{t}_{omega}"
            )


    # Warehouse capacity constraints
    # Retailer
    for r_val in R:
        for t in T:
            # Constraint (21)
            model.addConstr(
                gp.quicksum(vf_r[i_val, r_val, t, omega] for i_val in I) <= 0.9 * C_r[r_val],
                f"retailer_fwd_cap_C21_{r_val}_{t}_{omega}"
            )
            # Constraint (22)
            model.addConstr(
                gp.quicksum(vd_r[i_val, r_val, t, omega] for i_val in I) <= 0.1 * C_r[r_val],
                f"retailer_disposal_cap_C22_{r_val}_{t}_{omega}"
            )
    
    # Emergency
    for e_val in E:
        for t in T:
            # Constraint (23)
            model.addConstr(
                gp.quicksum(vf_e[i_val, e_val, t, omega] for i_val in I) <= 0.9 * C_e[e_val],
                f"emergency_fwd_cap_C23_{e_val}_{t}_{omega}"
            )
            # Constraint (24)
            model.addConstr(
                gp.quicksum(vd_e[i_val, e_val, t, omega] for i_val in I) <= 0.1 * C_e[e_val],
                f"emergency_disposal_cap_C24_{e_val}_{t}_{omega}"
            )
    
    # Provider
    for k_val in K:
        for t in T:
            sum_active_commitments_r = sum_active_commitments_r_precalculated[k_val, t]

            # Constraint (25)
            model.addConstr(
                gp.quicksum(vf_k[i_val, k_val, t, omega] for i_val in I) <=
                0.9 * C_k[k_val] * sum_active_commitments_r,
                f"provider_fwd_cap_C25_{k_val}_{t}_{omega}"
            )
            # Constraint (26)
            model.addConstr(
                gp.quicksum(vd_k[i_val, k_val, t, omega] for i_val in I) <=
                0.1 * C_k[k_val] * sum_active_commitments_r,
                f"provider_disposal_cap_C26_{k_val}_{t}_{omega}"
            )


In [244]:
model.update()
print("Total variables:", model.NumVars)
print("Total constraints:", model.NumConstrs)

Total variables: 1585
Total constraints: 1113


In [245]:
model.write("NewFormulationCode.lp")
model.write("NewFormulationCode.mps")

In [246]:
model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 1113 rows, 1585 columns and 5087 nonzeros
Model fingerprint: 0x25e75b22
Variable types: 1302 continuous, 283 integer (283 binary)
Coefficient statistics:
  Matrix range     [1e-01, 5e+03]
  Objective range  [7e-02, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+03]
Found heuristic solution: objective 3500.0000000
Presolve removed 461 rows and 356 columns
Presolve time: 0.01s
Presolved: 652 rows, 1229 columns, 3877 nonzeros
Variable types: 990 continuous, 239 integer (239 binary)

Root relaxation: objective 2.200000e+03, 282 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap 

In [247]:
if model.status == GRB.OPTIMAL:
    print(f"Optimal solution found with objective value: {model.objVal:.4f}")
    def safe_get_items(var_collection):
        if isinstance(var_collection, gp.tupledict):
            return var_collection.items()
        else:
            return [] 

    # Print selected suppliers (y_j)
    print("\nSelected Suppliers (y_j):")
    for j_key, y_var in safe_get_items(y):
        if y_var.X is not None and y_var.X > 0.5:
            print(f"  Supplier {j_key} is selected")
    
    # Print commitment decisions (g_kmt)
    print("\nCommitment Decisions (g_kmt):")
    for key, g_var in safe_get_items(g):
        k, m, t = key
        if g_var.X is not None and g_var.X > 0.5:
            print(f"  Warehouse {k}: {m}-period commitment starts at period {t}")
    
    # Print active commitments (r_kmt)
    print("\nActive Commitments (r_kmt):")
    for key, r_var in safe_get_items(r):
        k, m, t = key
        if r_var.X is not None and r_var.X > 0.5:
            print(f"  Warehouse {k}: {m}-period commitment is active at period {t}")

    # Print scenario-specific decisions
    for omega in Omega:
        print(f"\n--- Scenario {omega} ---")
        print("Transportation quantities (x_ijt):")
        
        # x_r
        for key, x_r_var in safe_get_items(x_r):
            i_val, j_val, r_val, t_val, omega_val = key
            if omega_val == omega and x_r_var.X is not None and x_r_var.X > 1e-6:
                if r_val == R[0] or len(R) == 1:
                        print(f"  Item {i_val}, Supplier {j_val} to Retailer {r_val} in Period {t_val}: {x_r_var.X:.2f}")
        
        # x_k
        for key, x_k_var in safe_get_items(x_k):
            i_val, j_val, k_val, t_val, omega_val = key
            if omega_val == omega and x_k_var.X is not None and x_k_var.X > 1e-6:
                if k_val == K[0] or len(K) == 1:
                        print(f"  Item {i_val}, Supplier {j_val} to Provider {k_val} in Period {t_val}: {x_k_var.X:.2f}")

        # x_e
        for key, x_e_var in safe_get_items(x_e):
            i_val, j_val, e_val, t_val, omega_val = key
            if omega_val == omega and x_e_var.X is not None and x_e_var.X > 1e-6:
                if e_val == E[0] or len(E) == 1:
                        print(f"  Item {i_val}, Supplier {j_val} to Emergency {e_val} in Period {t_val}: {x_e_var.X:.2f}")
        
        print("\nInventory levels (vf, vd):")
        for i_val in I:
            for t_val in T:
                print(f"  Item {i_val}, Period {t_val}:")
                # Retailer
                vf_r_val = 0
                vd_r_val = 0
                if isinstance(vf_r, gp.tupledict) and (i_val, R[0], t_val, omega) in vf_r.keys() and vf_r[i_val, R[0], t_val, omega].X is not None:
                    vf_r_val = vf_r[i_val, R[0], t_val, omega].X
                if isinstance(vd_r, gp.tupledict) and (i_val, R[0], t_val, omega) in vd_r.keys() and vd_r[i_val, R[0], t_val, omega].X is not None:
                    vd_r_val = vd_r[i_val, R[0], t_val, omega].X
                print(f"    Retailer ({R[0]}): Forward={vf_r_val:.2f}, Disposal={vd_r_val:.2f}")
                
                # Provider
                vf_k_val = 0
                vd_k_val = 0
                if isinstance(vf_k, gp.tupledict) and (i_val, K[0], t_val, omega) in vf_k.keys() and vf_k[i_val, K[0], t_val, omega].X is not None:
                    vf_k_val = vf_k[i_val, K[0], t_val, omega].X
                if isinstance(vd_k, gp.tupledict) and (i_val, K[0], t_val, omega) in vd_k.keys() and vd_k[i_val, K[0], t_val, omega].X is not None:
                    vd_k_val = vd_k[i_val, K[0], t_val, omega].X
                print(f"    Provider ({K[0]}): Forward={vf_k_val:.2f}, Disposal={vd_k_val:.2f}")
                
                # Emergency
                vf_e_val = 0
                vd_e_val = 0
                if isinstance(vf_e, gp.tupledict) and (i_val, E[0], t_val, omega) in vf_e.keys() and vf_e[i_val, E[0], t_val, omega].X is not None:
                    vf_e_val = vf_e[i_val, E[0], t_val, omega].X
                if isinstance(vd_e, gp.tupledict) and (i_val, E[0], t_val, omega) in vd_e.keys() and vd_e[i_val, E[0], t_val, omega].X is not None:
                    vd_e_val = vd_e[i_val, E[0], t_val, omega].X
                print(f"    Emergency ({E[0]}): Forward={vf_e_val:.2f}, Disposal={vd_e_val:.2f}")

        print("\nLost sales (z) and Unaccepted returns (w):")
        for key, z_var in safe_get_items(z):
            i_val, t_val, omega_val = key
            if omega_val == omega and z_var.X is not None and z_var.X > 1e-6:
                print(f"  Item {i_val}, Period {t_val}: Lost Sales = {z_var.X:.2f}")
        for key, w_var in safe_get_items(w):
            i_val, t_val, omega_val = key
            if omega_val == omega and w_var.X is not None and w_var.X > 1e-6:
                print(f"  Item {i_val}, Period {t_val}: Unaccepted Returns = {w_var.X:.2f}")
        
        print("\nRefurbished quantities (q):")
        for i_val in I:
            for t_val in T:
                q_r_list = []
                if isinstance(q_r, gp.tupledict):
                    q_r_list = [q_r[i_val, r_val, t_val, omega].X for r_val in R if (i_val, r_val, t_val, omega) in q_r.keys() and q_r[i_val, r_val, t_val, omega].X is not None]
                
                q_k_list = []
                if isinstance(q_k, gp.tupledict):
                    q_k_list = [q_k[i_val, k_val, t_val, omega].X for k_val in K if (i_val, k_val, t_val, omega) in q_k.keys() and q_k[i_val, k_val, t_val, omega].X is not None]
                
                q_e_list = []
                if isinstance(q_e, gp.tupledict):
                    q_e_list = [q_e[i_val, e_val, t_val, omega].X for e_val in E if (i_val, e_val, t_val, omega) in q_e.keys() and q_e[i_val, e_val, t_val, omega].X is not None]

                q_r_val = sum(q_r_list) if q_r_list else 0
                q_k_val = sum(q_k_list) if q_k_list else 0
                q_e_val = sum(q_e_list) if q_e_list else 0
                
                print(f"  Item {i_val}, Period {t_val}: Total Refurbished = {q_r_val + q_k_val + q_e_val:.2f}")
else:
    print(f"Optimization did not finish with an optimal solution. Status: {model.status}")

Optimal solution found with objective value: 3100.0000

Selected Suppliers (y_j):
  Supplier s2 is selected

Commitment Decisions (g_kmt):

Active Commitments (r_kmt):

--- Scenario omega1 ---
Transportation quantities (x_ijt):
  Item item1, Supplier s2 to Retailer r1 in Period 6: 100.00
  Item item1, Supplier s2 to Emergency e1 in Period 4: 100.00

Inventory levels (vf, vd):
  Item item1, Period 1:
    Retailer (r1): Forward=0.00, Disposal=0.00
    Provider (k1): Forward=0.00, Disposal=0.00
    Emergency (e1): Forward=0.00, Disposal=0.00
  Item item1, Period 2:
    Retailer (r1): Forward=0.00, Disposal=0.00
    Provider (k1): Forward=0.00, Disposal=0.00
    Emergency (e1): Forward=0.00, Disposal=0.00
  Item item1, Period 3:
    Retailer (r1): Forward=0.00, Disposal=0.00
    Provider (k1): Forward=0.00, Disposal=0.00
    Emergency (e1): Forward=0.00, Disposal=0.00
  Item item1, Period 4:
    Retailer (r1): Forward=0.00, Disposal=0.00
    Provider (k1): Forward=0.00, Disposal=0.00
    E

In [248]:
print(safe_get_items(g))

dict_items([(('k1', 1, 1), <gurobi.Var g[k1,1,1] (value 0.0)>), (('k1', 1, 2), <gurobi.Var g[k1,1,2] (value 0.0)>), (('k1', 1, 3), <gurobi.Var g[k1,1,3] (value 0.0)>), (('k1', 1, 4), <gurobi.Var g[k1,1,4] (value 0.0)>), (('k1', 1, 5), <gurobi.Var g[k1,1,5] (value 0.0)>), (('k1', 1, 6), <gurobi.Var g[k1,1,6] (value 0.0)>), (('k1', 1, 7), <gurobi.Var g[k1,1,7] (value 0.0)>), (('k1', 2, 1), <gurobi.Var g[k1,2,1] (value 0.0)>), (('k1', 2, 2), <gurobi.Var g[k1,2,2] (value -0.0)>), (('k1', 2, 3), <gurobi.Var g[k1,2,3] (value -0.0)>), (('k1', 2, 4), <gurobi.Var g[k1,2,4] (value -0.0)>), (('k1', 2, 5), <gurobi.Var g[k1,2,5] (value -0.0)>), (('k1', 2, 6), <gurobi.Var g[k1,2,6] (value -0.0)>), (('k1', 2, 7), <gurobi.Var g[k1,2,7] (value -0.0)>), (('k1', 3, 1), <gurobi.Var g[k1,3,1] (value 0.0)>), (('k1', 3, 2), <gurobi.Var g[k1,3,2] (value -0.0)>), (('k1', 3, 3), <gurobi.Var g[k1,3,3] (value -0.0)>), (('k1', 3, 4), <gurobi.Var g[k1,3,4] (value -0.0)>), (('k1', 3, 5), <gurobi.Var g[k1,3,5] (value